# <center> R1.13 Introduction à Python <br> TP4 - Nim, Marienbad et IA </center>
<center> 2024/2025 - CARRÉ Lucien, COIGNARD Maël, COUDIERE Adrien - Groupe D </center>
<center> IUT de Vannes, BUT Informatique </center>

***

# <center> Jeu de Marienbad </center>

In [ ]:
import random

################### Message de bienvenue et d'explication ##################

def message_bienvenue():
    print("""
----------------------------------------------------------------------------
                        Règles du jeu de Marienbad
----------------------------------------------------------------------------
Règles : 1 - Deux joueurs s'affrontent à tour de rôle.
         2 - L'ordinateur commence.
         3 - Le joueur peut retirer un ou plusieurs bâtons, mais seulement dans une seule rangée.
         4 - Celui qui prend la dernière allumette gagne la partie.
----------------------------------------------------------------------------
                           Modes de jeu
----------------------------------------------------------------------------                        
1 - Humain contre Ordinateur
2 - Ordinateur contre Hasard
3 - Ordinateur contre Ordinateur
4 - Humain contre Humain
----------------------------------------------------------------------------
                           Apprentissage
----------------------------------------------------------------------------
Suivant qu'il gagne ou perd, l'ordinateur reçoit une récompense ou une punition.
----------------------------------------------------------------------------
""")

################### Affichage du plateau ##################

def affiche(plateau):
    print("\nPlateau actuel :")
    for i, rangee in enumerate(plateau):
        print(f"Rangée {i + 1}: {'| ' * rangee} ({rangee})")
    print()

################### Initialisation du jeu de Marienbad ##################

### Initialisation du plateau ###
def configurer_plateau():
    while True:
        try:
            nb_rangees = int(input("Entrez le nombre de rangées (entre 2 et 6) : "))
            if nb_rangees <= 1 or nb_rangees >= 7:
                print("Le nombre de rangées doit être compris entre 2 et 6.")
            else:
                return [i * 2 - 1 for i in range(1, nb_rangees + 1)]
        except ValueError:
            print("Entrée invalide. Veuillez entrer un entier positif.")

### Fait un coup aléatoire ###
def coup_aleatoire(plateau_actuel):
    rangees_valides = [i for i, rangee in enumerate(plateau_actuel) if rangee > 0]
    rangee = random.choice(rangees_valides)
    allumettes = random.randint(1, plateau_actuel[rangee])
    return rangee, allumettes

### Crée un dictionnaire pour tous les coups possibles pour chaque état possible du plateau ###
def possibilites(plateau):
    copie_plateau = [0 for _ in range(len(plateau))]
    dico_possibilites = {}
    
    while copie_plateau != plateau:
        copie_plateau[-1] += 1
        
        for i in range(len(copie_plateau) - 1, 0, -1):
            if copie_plateau[i] == plateau[i] + 1:
                copie_plateau[i] = 0
                copie_plateau[i - 1] += 1
        
            dico_possibilites[str(copie_plateau)] = {}
            for i in range(len(copie_plateau)):
                for j in range(copie_plateau[i]):
                    dico_possibilites[str(copie_plateau)][i, j + 1] = 15
    
    return dico_possibilites

### Le coup de l'ordinateur ###
def coup_ordi(plateau_actuel, historique, dico_possibilites):
    leCoup = random.choices(list(dico_possibilites[str(plateau_actuel)].keys()), weights = dico_possibilites[str(plateau_actuel)].values())
    historique.append([str(plateau_actuel), leCoup[0]])

    return leCoup[0][0], leCoup[0][1]

################### Programme principal ##################

message_bienvenue()
mode = 0

# Boucle pour le choix du mode de jeux
while mode not in [1, 2, 3, 4]:
    try:
        mode = int(input("\nChoisissez un mode de jeu (1, 2, 3 ou 4) : "))
        if mode not in [1, 2, 3, 4]:
            print("Mode invalide. Veuillez choisir entre 1, 2, 3 ou 4.")
    except ValueError:
        print("Entrée invalide. Veuillez saisir un nombre.")

plateau = configurer_plateau()

if mode == 2 or mode == 3:
    nb_parties_a_jouer = int(input("\nChoisissez le nombre de parties à jouer : "))

memoire_cmp = possibilites(plateau)
if mode == 3:
    memoire_cmp2 = possibilites(plateau)

if mode == 4:
    nom_joueur1 = input("Entrez le nom du premier joueur : ")
    nom_joueur2 = input("Entrez le nom du deuxième joueur : ")


uneautrepartie = True
compteur_parties = 0 
score = [0, 0]

# Boucle principale pour le déroulé d'une partie 
while uneautrepartie:
    if mode == 1:
        joueur = 'Ordinateur'
    elif mode == 2:
        joueur = random.choice(['Ordinateur', 'Aleatoire'])
    elif mode == 3:
        joueur = random.choice(['Ordinateur', 'Ordinateur 2'])
    elif mode == 4:
        joueur = random.choice([nom_joueur1, nom_joueur2])

    plateau_actuel = plateau[:]

    historique_cmp = []
    if mode == 3:
        historique_cmp2 = []

    # Boucle pour le déroulé de la partie (tant qu'il reste des allumettes)
    while sum(plateau_actuel) > 0:
        affiche(plateau_actuel)

        if joueur == 'Ordinateur':  # Tour de l'ordinateur
            print("\nL'ordinateur choisit de jouer...")
            rangee, allumettes = coup_ordi(plateau_actuel, historique_cmp, memoire_cmp)
            plateau_actuel[rangee] -= allumettes
            print(f"L'ordinateur enlève {allumettes} allumette(s) de la rangée {allumettes + 1}.")

            if sum(plateau_actuel) == 0:
                vainqueur = 'Ordinateur'
            else:
                joueur = 'Utilisateur' if mode == 1 else 'Aleatoire' if mode == 2 else 'Ordinateur 2'

        elif joueur == 'Utilisateur':  # Tour de l'utilisateur
            print("\n-- À vous de jouer ! --")
            coup_valide = False

            while not coup_valide:
                try:
                    rangee = int(input("Choisissez une rangée : ")) - 1
                    allumettes = int(input("Combien d'allumettes voulez-vous enlever ? "))

                    if rangee < 0 or rangee >= len(plateau_actuel) or plateau_actuel[rangee] < allumettes or allumettes <= 0:
                        print("Coup invalide. Recommencez.")
                    else:
                        coup_valide = True
                except ValueError:
                    print("Entrée invalide. Recommencez.")

            plateau_actuel[rangee] -= allumettes
            print(f"Vous avez retiré {allumettes} allumette(s) de la rangée {rangee + 1}.")

            if sum(plateau_actuel) == 0:
                vainqueur = 'Utilisateur'
            else:
                joueur = 'Ordinateur'

        elif joueur == 'Aleatoire':  # Tour d'un joueur aléatoire
            print("\nLe joueur au hasard choisit de jouer...")
            rangee, allumettes = coup_aleatoire(plateau_actuel)
            plateau_actuel[rangee] -= allumettes
            print(f"Le joueur au hasard enlève {allumettes} allumette(s) de la rangée {rangee + 1}.")

            if sum(plateau_actuel) == 0:
                vainqueur = 'Aleatoire'
            else:
                joueur = 'Ordinateur'

        elif joueur == 'Ordinateur 2':  # Deuxième IA dans le mode ordinateur contre ordinateur
            print("\nL'autre ordinateur choisit de jouer...")
            rangee, allumettes = coup_ordi(plateau_actuel, historique_cmp2, memoire_cmp2)
            plateau_actuel[rangee] -= allumettes
            print(f"L'autre ordinateur enlève {allumettes} allumette(s) de la rangée {rangee + 1}.")

            if sum(plateau_actuel) == 0:
                vainqueur = 'Ordinateur 2'
            else:
                joueur = 'Ordinateur'

        elif joueur == nom_joueur1:  # Tour du premier joueur humain dans le mode JvJ
            print(f"\n-- {nom_joueur1}, à vous de jouer ! --")
            coup_valide = False

            while not coup_valide:
                try:
                    rangee = int(input("Choisissez une rangée : ")) - 1
                    allumettes = int(input("Combien d'allumettes voulez-vous enlever ? "))

                    if allumettes < 0 or allumettes >= len(plateau_actuel) or plateau_actuel[allumettes] < allumettes or allumettes <= 0:
                        print("Coup invalide. Recommencez.")
                    else:
                        coup_valide = True
                except ValueError:
                    print("Entrée invalide. Recommencez.")

            plateau_actuel[rangee] -= allumettes
            print(f"{nom_joueur1} a retiré {allumettes} allumette(s) de la rangée {rangee + 1}.")

            if sum(plateau_actuel) == 0:
                vainqueur = nom_joueur1
            else:
                joueur = nom_joueur2

        elif joueur == nom_joueur2:  # Tour du deuxième joueur humain dans le mode JvJ
            print(f"\n-- {nom_joueur2}, à vous de jouer ! --")
            coup_valide = False

            while not coup_valide:
                try:
                    rangee = int(input("Choisissez une rangée : ")) - 1
                    allumettes = int(input("Combien d'allumettes voulez-vous enlever ? "))

                    if rangee < 0 or rangee >= len(plateau_actuel) or plateau_actuel[rangee] < allumettes or allumettes <= 0:
                        print("Coup invalide. Recommencez.")
                    else:
                        coup_valide = True
                except ValueError:
                    print("Entrée invalide. Recommencez.")

            plateau_actuel[rangee] -= allumettes
            print(f"{nom_joueur2} a retiré {allumettes} allumette(s) de la rangée {rangee + 1}.")

            if sum(plateau_actuel) == 0:
                vainqueur = nom_joueur2
            else:
                joueur = nom_joueur2

    compteur_parties += 1

    

    # Annonce des résultats
    if vainqueur == 'Ordinateur':
        print("\n----------------------------------------------------------------------------")
        print("L'ordinateur a gagné en ramassant la dernière allumette.")
        print("----------------------------------------------------------------------------\n")
        if mode == 1:
            score[1] += 1
        else:
            score[0] += 1

    elif vainqueur == 'Utilisateur':
        print("\n----------------------------------------------------------------------------")
        print("Bravo ! Vous avez gagné en ramassant la dernière allumette.")
        print("----------------------------------------------------------------------------\n")
        score[0] += 1

    elif vainqueur == 'Aleatoire':
        print("\n----------------------------------------------------------------------------")
        print("Le joueur au hasard a gagné en ramassant la dernière allumette.")
        print("----------------------------------------------------------------------------\n")
        score[1] += 1

    elif vainqueur == 'Ordinateur 2':
        print("\n----------------------------------------------------------------------------")
        print("L'autre ordinateur a gagné en ramassant la dernière allumette.")
        print("----------------------------------------------------------------------------\n")
        score[1] += 1

    elif vainqueur == nom_joueur1:
        print("\n----------------------------------------------------------------------------")
        print(f"{nom_joueur1} a gagné en ramassant la dernière allumette.")
        print("----------------------------------------------------------------------------\n")
        score[0] += 1

    elif vainqueur == nom_joueur2:
        print("\n----------------------------------------------------------------------------")
        print(f"{nom_joueur2} a gagné en ramassant la dernière allumette.")
        print("----------------------------------------------------------------------------\n")
        score[1] += 1

    # Récompense ou punition
    if vainqueur == 'Ordinateur':
        for etat, coup in historique_cmp:
            memoire_cmp[etat][coup] += 5
        if mode == 3 : 
            for etat, coup in historique_cmp2:
                memoire_cmp2[etat][coup] -= 3
    else:
        if mode == 3:
            for etat, coup in historique_cmp2:
                memoire_cmp2[etat][coup] += 5
        for etat, coup in historique_cmp:
            memoire_cmp[etat][coup] -= 3

    # Réinitialisation des mémoires vides
    for etat in memoire_cmp:
        for coup in memoire_cmp[etat]:
            if memoire_cmp[etat][coup] <= 0:
                memoire_cmp[etat][coup] = 1
    if mode == 3:
        for etat in memoire_cmp2:
            for coup in memoire_cmp2[etat]:
                if memoire_cmp2[etat][coup] <= 0:
                    memoire_cmp2[etat][coup] = 1

    
    """ # Calcul des probabilités de victoire à la prochaine partie pour le ou les ordinateurs
    probabilite_cmp = calcul_probabilites()
    if mode == 3:
        probabilite_cmp2 = calcul_probabilites() """
    
    print("\n----------------------------------------------------------------------------")
    print("Vous avez joué "+ str(compteur_parties) +" parties.")
    print("----------------------------------------------------------------------------\n") 
    if mode == 1:
        print("\n----------------------------------------------------------------------------")
        print("Score : ")
        print("Joueur : " + str(score[0]) + " (" + str(round((score[0]/compteur_parties) * 100, 2)) + " %)")
        print("Ordinateur : " + str(score[1]) + " (" + str(round((score[1]/compteur_parties) * 100, 2)) + " %)")
        print("----------------------------------------------------------------------------\n")
    elif mode == 2:
        print("\n----------------------------------------------------------------------------")
        print("Score : ")
        print("Ordinateur : " + str(score[0]) + " (" + str(round((score[0]/compteur_parties) * 100, 2)) + " %)")
        print("Hasard : " + str(score[1]) + " (" + str(round((score[1]/compteur_parties) * 100, 2)) + " %)")
        print("----------------------------------------------------------------------------\n")
    elif mode == 3:
        print("\n----------------------------------------------------------------------------")
        print("Score : ")
        print("Ordinateur 1 : " + str(score[0]) + " (" + str(round((score[0]/compteur_parties) * 100, 2)) + " %)")
        print("Ordinateur 2 : " + str(score[1]) + " (" + str(round((score[1]/compteur_parties) * 100, 2)) + " %)")
        print("----------------------------------------------------------------------------\n")
    elif mode == 4:
        print("\n----------------------------------------------------------------------------")
        print("Score : ")
        print(str(nom_joueur1) + " : " + str(score[0]) + " (" + str(round((score[0]/compteur_parties) * 100, 2)) + " %)")
        print(str(nom_joueur2) + " : " + str(score[1]) + " (" + str(round((score[1]/compteur_parties) * 100, 2)) + " %)")
        print("----------------------------------------------------------------------------\n")

    # Continuer ou arrêter ?
    if mode == 1 or mode == 4:
        test = True
        while test:
            nouvelle_partie = input("\nVoulez-vous rejouer ? [O/N]: ")
            if nouvelle_partie in ("o", "O"):
                uneautrepartie = True
                test = False
            elif nouvelle_partie in ("n", "N"):
                uneautrepartie = False
                test = False
            else:
                print("\nChoix invalide. Recommencez !")
    else:
        if compteur_parties == nb_parties_a_jouer:
            uneautrepartie = False
        else:
            uneautrepartie = True